In [ ]:
import re
import math
import sys
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from decimal import Decimal, ROUND_HALF_UP

In [ ]:
def parse_line(string):
    attribute = re.findall('(?<=\[)[A-z0-9]+', string)[0]
    text = re.findall('(?<=\").+(?=\")', string)[0]
    return attribute, text

def add_dic(complete, current):
    current['Rating Difference'] = current['White']-current['Black']
    if current['Rating Difference'] in complete[current['Game Speed']]:
        if current['Winner'] == .5:
            complete[current['Game Speed']][current['Rating Difference']] = [sum(x) for x in zip(complete[current['Game Speed']][current['Rating Difference']], [current['Winner'], 1, 1])]
        else:
            complete[current['Game Speed']][current['Rating Difference']] = [sum(x) for x in zip(complete[current['Game Speed']][current['Rating Difference']], [current['Winner'], 1, 0])]
    else:
        if current['Winner'] == .5:
            complete[current['Game Speed']][current['Rating Difference']] = [current['Winner'], 1, 1]
        else:
            complete[current['Game Speed']][current['Rating Difference']] = [current['Winner'], 1, 0]

In [ ]:
complete = {'Bullet':{}, 'Blitz':{}, 'Rapid':{}, 'Classical':{}}
total = 0
terminations = []

with open(r"..\..\data\lichess-june.pgn") as f:
    nextLine = f.readline()
    current = {}
    while nextLine != '':
        if nextLine.startswith("["):
            print(text)
            attribute, text = parse_line(nextLine)
            
            if attribute == 'WhiteElo':
                current['White'] = int(text)
            
            if attribute == 'BlackElo':
                current['Black'] = int(text)
            
            if attribute == 'Event':
                try:
                    current['Game Speed'] = next(perf for perf in ['Rapid', 'Bullet', 'Blitz', 'Classical'] if perf in text)
                except:
                    pass
            
            if attribute == 'Result':
                try:
                    winner = eval(re.findall("[0-9/]+(?=-)", text)[0])
                    current['Winner'] = winner
                except:
                    pass
                
            if attribute == 'Termination':
                if text not in terminations:
                    terminations.append(text)
                if set(['Winner', 'Game Speed', 'White', 'Black']) == set(current.keys()):
                    dif = current['White']-current['Black']
                    add_dic(complete, current)
                    
                    total += 1
                    if total%100000 == 0:
                        print(total)
                    current = {}
        
        nextLine = f.readline()

In [ ]:
results = []

for k in complete:
    for l in complete[k]:
        results.append([k] + [l] + complete[k][l])
        
results = pd.DataFrame(results, columns=['Game Speed', 'Rating Difference', 'Total Points', 'Total Games', 'Total Draws'])
results['Game Speed'] = pd.Categorical(results['Game Speed'], ["Bullet", "Blitz", "Rapid", "Classical"])
results.sort_values(by=['Game Speed', 'Rating Difference'], inplace=True)

results.to_csv(r'..\..\data\baseline-june.csv', index=False)